In [1]:
import numpy as np
import tensorflow as tf

In [2]:
#What are Tensors?
"""
 Tensors are multilinear maps from vector spaces to the real numbers : 
         A scalar is a tensor; A vector is a tensor; A matrix is a tensor
"""
""""""

''

In [3]:
#Is TensorFlow any similar to NUMPY?
"""
TensorFlow and Numpy are quite similar:
    Both are N-d array libraries!
    Numpy has Ndarray support, but doesn’t offer methods to create tensor functions and automatically compute 
    derivatives (+ no GPU support)

Resemblance to the sexy NUMPY
    np.zeros()~~> tf.zeros()
    np.array([[1.0, 2.0], [3.0, 4.0]], dtype=np.float32)  ~~> tf.constant([[1.0, 2.0], [3.0, 4.0]])

"""
""""""

''

In [4]:
a = np.zeros((2,2)); 
b = np.ones((2,2))
c=np.array([[1,2],[3,4]])
print(np.sum(b, axis=0))
print(np.sum(b, axis=1))
print(np.sum(b))
print(a.shape)
print(np.reshape(a, (1,4)))
print(b*4+3)
print(np.dot(b,c))#[0][0]=[1,1]*[1,3]=4;[0][1]=[1,1]*[2,4]=6 [Matrix Multiplication]

[ 2.  2.]
[ 2.  2.]
4.0
(2, 2)
[[ 0.  0.  0.  0.]]
[[ 7.  7.]
 [ 7.  7.]]
[[ 4.  6.]
 [ 4.  6.]]


In [5]:
tf.InteractiveSession()#just convenient syntactic sugar for keeping a default session open in ipython
a = tf.zeros((2,2)); 
b = tf.ones((2,2))
c=tf.constant([[1,2],[3,4]],dtype=tf.float32)#By Default the dtype is int32
print(tf.reduce_sum(b, reduction_indices=0).eval())
print(tf.reduce_sum(b, reduction_indices=1).eval())
print(tf.reduce_sum(b).eval())
print(a.get_shape())
print(tf.reshape(a, (1, 4)).eval())
print((b*4+3).eval())
print(tf.matmul(b,c).eval())

[ 2.  2.]
[ 2.  2.]
4.0
(2, 2)
[[ 0.  0.  0.  0.]]
[[ 7.  7.]
 [ 7.  7.]]
[[ 4.  6.]
 [ 4.  6.]]


In [6]:
#Converting Python Objects to TensorFlow
def python_to_tensor(python_obj):
    tf_tensor = tf.convert_to_tensor(python_obj, dtype=tf.float32)
    return  tf.matmul(tf_tensor,tf.matrix_transpose(tf_tensor))#Are you expecting a Symmetric matrix :-)
# The following calls are equivalent.
value_1 = python_to_tensor(tf.constant([[1.0, 2.0], [3.0, 4.0]]))
value_2 = python_to_tensor([[1.0, 2.0], [3.0, 4.0]])
value_3 = python_to_tensor(np.array([[1.0, 2.0], [3.0, 4.0]], dtype=np.float32))
value_1.eval(),value_2.eval(),value_3.eval()

(array([[  5.,  11.],
        [ 11.,  25.]], dtype=float32), array([[  5.,  11.],
        [ 11.,  25.]], dtype=float32), array([[  5.,  11.],
        [ 11.,  25.]], dtype=float32))

In [7]:
#TensorFlow requires explicit evaluation!
a = np.zeros((2,2))
ta = tf.zeros((2,2))
print(a)
print(ta)
print(ta.eval())#ensorFlow computations define a computation graph that has no numerical value until evaluated

[[ 0.  0.]
 [ 0.  0.]]
Tensor("zeros_1:0", shape=(2, 2), dtype=float32)
[[ 0.  0.]
 [ 0.  0.]]


In [8]:
#A Session object encapsulates the environment in which  Tensor objects are evaluated
a=tf.constant(5)
b=tf.constant(7)
c=a*b
with tf.Session() as sess:
    print(sess.run(c))
    print(c.eval())#Just a syntactic sugar for sess.run(c)

35
35


In [9]:
#All computations add nodes to global default graph : Tensor Flow Computation Graph

In [10]:
"""
TF Variables:
    In-Memory Buffers which Contain Tensors
    Frequently used to Hold Model Hyper Parameters
"""
""""""

''

In [11]:
w1=tf.ones((2,2))
w2=tf.Variable(tf.ones((2,2)),name="wts")
with tf.Session() as sess:
    print(sess.run(w1))
    #print(sess.run(w2))#FailedPreconditionError: Attempting to use uninitialized value wts_1
    sess.run(tf.global_variables_initializer())
    print(sess.run(w2))

[[ 1.  1.]
 [ 1.  1.]]
[[ 1.  1.]
 [ 1.  1.]]


In [12]:
#Updating Variable State:
a=tf.constant(3)
b=tf.constant(2)
c=tf.multiply(a,b)
var1=tf.Variable(0,name="i")
incr=tf.add(var1,tf.constant(1),name="incr")#Takes two tensor & sums them element wise to produce another tensor
update_var1=tf.assign(var1,incr)#Will assign the value "incr" to  var1; Returns a (New ?)Tensor with the updated value
with tf.Session() as addSession:
    addSession.run(tf.global_variables_initializer())
    print(">>>>>>>>>>><<<<<<<<<<<<<<")
    print(addSession.run(c))
    for _ in range(3):
        addSession.run(update_var1)
        print(addSession.run(var1),addSession.run([var1,incr,update_var1]))

>>>>>>>>>>><<<<<<<<<<<<<<
6
1 [2, 2, 2]
3 [4, 4, 4]
5 [6, 6, 6]


In [13]:
#importing Data from Numpy
a=np.zeros((2,3))
ta=tf.convert_to_tensor(a)
#Inputting data with tf.convert_to_tensor() is  convenient, but doesn’t scale.
with tf.Session() as sess:
    print(sess.run(ta))

[[ 0.  0.  0.]
 [ 0.  0.  0.]]


In [14]:
#Use tf.placeholder variables(Dummy Nodes: Entry point for Data to Computational Graph)
#feed_dict: Python Dictionary to map from tf.placeholder to data (numpy arrays, lists etc.)
ip1=tf.placeholder(tf.float32)
ip2=tf.placeholder(tf.float32)
op=tf.multiply(ip1,ip2)
with tf.Session() as sess:
    #print(sess.run(op))# You must feed a value for placeholder tensor
    print(sess.run(op,feed_dict={ip1:12,ip2:8}))
    print(sess.run([op,ip1,ip2],feed_dict={ip1:12,ip2:8}))
    print(sess.run([op,ip1,ip2],feed_dict={ip1:[12],ip2:[8]}))
    print(sess.run([op,ip1,ip2],feed_dict={ip1:[12,10],ip2:[8,7]}))
    print(sess.run([op,ip1,ip2],feed_dict={ip1:np.array([12,10]),ip2:np.array([8,7])}))

96.0
[96.0, array(12.0, dtype=float32), array(8.0, dtype=float32)]
[array([ 96.], dtype=float32), array([ 12.], dtype=float32), array([ 8.], dtype=float32)]
[array([ 96.,  70.], dtype=float32), array([ 12.,  10.], dtype=float32), array([ 8.,  7.], dtype=float32)]
[array([ 96.,  70.], dtype=float32), array([ 12.,  10.], dtype=float32), array([ 8.,  7.], dtype=float32)]


In [15]:
test_init_graph = tf.Graph()
with test_init_graph.as_default():
    test_scalar_var = tf.Variable(2)
    incr_scalar=tf.add(test_scalar_var,1)
    update_scalar=tf.assign(test_scalar_var,incr_scalar)

In [16]:
with tf.Session(graph=test_init_graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for i in range(4):
        my_scalar_val=session.run([update_scalar])
        print(my_scalar_val)

Initialized
[3]
[4]
[5]
[6]


In [17]:
with tf.Session(graph=test_init_graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for i in range(7):
        test_scalar_val=session.run([test_scalar_var])
        print("Inital Val of Variable ",test_scalar_val)
        incr_scalar_val=session.run([incr_scalar])
        print("To be Assigned Value is ",incr_scalar_val)
        my_scalar_val=session.run([update_scalar])
        print("Updated Val of Variable ",my_scalar_val)

Initialized
Inital Val of Variable  [2]
To be Assigned Value is  [3]
Updated Val of Variable  [3]
Inital Val of Variable  [3]
To be Assigned Value is  [4]
Updated Val of Variable  [4]
Inital Val of Variable  [4]
To be Assigned Value is  [5]
Updated Val of Variable  [5]
Inital Val of Variable  [5]
To be Assigned Value is  [6]
Updated Val of Variable  [6]
Inital Val of Variable  [6]
To be Assigned Value is  [7]
Updated Val of Variable  [7]
Inital Val of Variable  [7]
To be Assigned Value is  [8]
Updated Val of Variable  [8]
Inital Val of Variable  [8]
To be Assigned Value is  [9]
Updated Val of Variable  [9]


In [18]:
test_inits_graph = tf.Graph()
with test_inits_graph.as_default():
    test_scalar_var = tf.Variable(2)
    other_scalar_var = tf.Variable(2)
    incr_scalar=tf.add(test_scalar_var,1)
    update_scalar=tf.assign(test_scalar_var,incr_scalar)
    other_scalar=tf.assign(other_scalar_var,incr_scalar)

In [19]:
with tf.Session(graph=test_inits_graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for i in range(3):
        my_scalar_val=session.run([update_scalar])
        print("Updated Val of Variable ",my_scalar_val)
        other_scalar_val=session.run([other_scalar_var])
        print("Persistent Value of Variable ",other_scalar_val)

Initialized
Updated Val of Variable  [3]
Persistent Value of Variable  [2]
Updated Val of Variable  [4]
Persistent Value of Variable  [2]
Updated Val of Variable  [5]
Persistent Value of Variable  [2]


In [20]:
with tf.Session(graph=test_inits_graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for i in range(8):
        my_scalar_val=session.run([update_scalar])
        print("Updated Val of Variable ",my_scalar_val)
        if i%3==0 or i%3==1:
            other_scalar_val=session.run([other_scalar_var])
        else:
            other_scalar_val,_=session.run([other_scalar_var,other_scalar])
        print("Persistent Value of Variable ",other_scalar_val)

Initialized
Updated Val of Variable  [3]
Persistent Value of Variable  [2]
Updated Val of Variable  [4]
Persistent Value of Variable  [2]
Updated Val of Variable  [5]
Persistent Value of Variable  6
Updated Val of Variable  [6]
Persistent Value of Variable  [6]
Updated Val of Variable  [7]
Persistent Value of Variable  [6]
Updated Val of Variable  [8]
Persistent Value of Variable  9
Updated Val of Variable  [9]
Persistent Value of Variable  [9]
Updated Val of Variable  [10]
Persistent Value of Variable  [9]
